<a href="https://colab.research.google.com/github/rahiakela/fluent-python-book-practice/blob/master/part-iv-object-oriented-idioms/9_pythonic_object.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# A Pythonic object

Thanks to the Python data model, your user-defined types can behave as naturally as the built-in types. And this can be accomplished without inheritance, in the spirit of duck typing: you just implement the methods needed for your objects to behave as expected.



## Object representations

Every object-oriented language has at least one standard way of getting a string representation from any object.

- **repr()**: Return a string representing the object as the developer wants to see it.
- **str()**: Return a string representing the object as the user wants to see it.

As you know, we implement the special methods __repr__ and __str__ to support repr() and str().

## Vector class redux

To demonstrate the many methods used to generate object representations, we’ll use a 2D Vector2d class.At this point, Vector2d uses several
special methods to provide operations that a Pythonista expects in a well designed object.

In [15]:
from array import array
import math

In [16]:
class Vector2D:
  # typecode is a class attribute we’ll use when converting Vector2d instances to/from bytes.
  typecode = 'd'

  def __init__(self, x, y):
    # Converting x and y to float in __init__ catches errors early
    self.x = float(x)
    self.y = float(y)

  def __iter__(self):
    # makes it iterable; this is what makes unpacking work, e.g, x, y = my_vector.
    return (i for i in (self.x, self.y))

  def __repr__(self):
    class_name = type(self).__name__
    # builds a string by interpolating the components with {!r} to get their
    # repr; because Vector2d is iterable, *self feeds the x and y components to format.
    return '{}({!r}, {!r})'.format(class_name, *self)

  def __str__(self):
    # From an iterable Vector2d it’s easy to build a tuple for display as an ordered pair.
    return str(tuple(self))

  def __bytes__(self):
    # To generate bytes, we convert the typecode to bytes and concatenate…
    # …bytes converted from an array built by iterating over the instance.
    return (bytes([ord(self.typecode)]) + bytes(array(self.typecode, self)))

  def __eq__(self, other):
    # To quickly compare all components, build tuples out of the operands.
    return tuple(self) == tuple(other)

  def __abs__(self):
    # The magnitude is the length of the hypotenuse of the triangle formed by the x and y components.
    return math.hypot(self.x, self.y)

  def __bool__(self):
    # uses abs(self) to compute the magnitude, then converts it to bool
    return bool(abs(self))

The basic behavior we expect from a Vector2d instance.Vector2d instances have several representations.

In [17]:
v1 = Vector2D(3, 4)
# The components of a Vector2d can be accessed directly as attributes (no getter method calls).
print(v1.x, v1.y)

3.0 4.0


In [18]:
# A Vector2d can be unpacked to a tuple of variables.
x, y = v1
x, y

(3.0, 4.0)

In [19]:
# The repr of a Vector2d emulates the source code for constructing the instance.
v1

Vector2D(3.0, 4.0)

In [20]:
# Using eval here shows that the repr of a Vector2d is a faithful representation of its constructor call
v1_clone = eval(repr(v1))

# Vector2d supports comparison with ==; this is useful for testing.
v1 == v1_clone

# print calls str, which for Vector2d produces an ordered pair display
print(v1)

(3.0, 4.0)


In [21]:
# bytes uses the __bytes__ method to produce a binary representation.
octets = bytes(v1)
octets

b'd\x00\x00\x00\x00\x00\x00\x08@\x00\x00\x00\x00\x00\x00\x10@'

In [22]:
# abs uses the __abs__ method to return the magnitude of the Vector2d.
abs(v1)

5.0

In [23]:
# bool uses the __bool__ method to return False for a Vector2d of zero magnitude or True otherwise.
bool(v1), bool(Vector2D(0, 0))

(True, False)

## An alternative constructor

We have a fairly complete set of basic methods, but one obvious operation is missing: rebuilding a Vector2d from the binary representation produced by bytes().

